In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("setup").getOrCreate()

In [2]:
#read_files
transactions = spark.read.options(header='True', inferSchema='True', delimiter=';') \
     .csv("/home/jovyan/transactions.csv")

In [3]:
transactions.createOrReplaceTempView("transactions")

In [16]:
spark.sql("""
    WITH ranked_temp(
    SELECT *, ROUND(PERCENT_RANK() OVER(
                  ORDER BY amount), 2) AS Percent_Rank
    FROM transactions
    )
    SELECT 
        id, amount, account_type, transaction_type, country,
        CASE 
            WHEN Percent_rank >= 0.75 THEN 'high' 
            WHEN Percent_rank >= 0.25 AND Percent_rank < 0.75 THEN 'average'
            ELSE 'low'
        END AS level
    FROM
        ranked_temp
""").show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `transaction_type` cannot be resolved. Did you mean one of the following? [`ranked_temp`.`id`, `ranked_temp`.`account_type`, `ranked_temp`.`amount`, `ranked_temp`.`country`, `ranked_temp`.`transaction_date`].; line 8 pos 34;
'WithCTE
:- CTERelationDef 7, false
:  +- SubqueryAlias ranked_temp
:     +- Project [id#17, amount#18, account_type#19, transaction_date#20, country#21, Percent_Rank#361]
:        +- Project [id#17, amount#18, account_type#19, transaction_date#20, country#21, _we0#367, round(_we0#367, 2) AS Percent_Rank#361]
:           +- Window [percent_rank(amount#18) windowspecdefinition(amount#18 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS _we0#367], [amount#18 ASC NULLS FIRST]
:              +- Project [id#17, amount#18, account_type#19, transaction_date#20, country#21]
:                 +- SubqueryAlias transactions
:                 

In [ ]:
id, amount, account_type, transaction_type, country,
        CASE 
            WHEN Percent_rank >= 0.75 THEN 'high' 
            WHEN Percent_rank >= 0.25 AND Percent_rank < 0.75 THEN 'average'
            ELSE 'low'
        END AS level